# First Connect to the Wifi

In [ ]:
from pynq.lib import Wifi
port = Wifi()

In [ ]:
ssid = ###
pwd = ###
port.connect(ssid, pwd, auto=True, force=True)

In [ ]:
! ping -I wlan0 www.yahoo.com -c 5

# Set up Object Classification

In [ ]:
from pynq_dpu import DpuOverlay
overlay = DpuOverlay("/home/xilinx/DPU-PYNQ-3.5/dpu.bit")

In [ ]:
overlay?

In [ ]:
import os
import time
import numpy as np
import cv2
import random
import colorsys
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
overlay.load_model("/home/xilinx/DPU-PYNQ-3.5/tf_yolov3_3.5.xmodel")

In [ ]:
anchor_list = [10,13,16,30,33,23,30,61,62,45,59,119,116,90,156,198,373,326]
anchor_float = [float(x) for x in anchor_list]
anchors = np.array(anchor_float).reshape(-1, 2)

In [ ]:
'''Get model classification information'''
def get_class(classes_path):
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

classes_path = "pynq_dpu/img/voc_classes.txt"
class_names = get_class(classes_path)

In [ ]:
num_classes = len(class_names)
hsv_tuples = [(1.0 * x / num_classes, 1., 1.) for x in range(num_classes)]
colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
colors = list(map(lambda x:
                  (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                  colors))
random.seed(0)
random.shuffle(colors)
random.seed(None)

In [ ]:
def letterbox_image(image, size):
    ih, iw, _ = image.shape
    w, h = size
    scale = min(w/iw, h/ih)
    #print(scale)

    nw = int(iw*scale)
    nh = int(ih*scale)
    #print(nw)
    #print(nh)

    image = cv2.resize(image, (nw,nh), interpolation=cv2.INTER_LINEAR)
    new_image = np.ones((h,w,3), np.uint8) * 128
    h_start = (h-nh)//2
    w_start = (w-nw)//2
    new_image[h_start:h_start+nh, w_start:w_start+nw, :] = image
    return new_image


'''image preprocessing'''
def pre_process(image, model_image_size):
    image = image[...,::-1]
    image_h, image_w, _ = image.shape

    if model_image_size != (None, None):
        assert model_image_size[0]%32 == 0, 'Multiples of 32 required'
        assert model_image_size[1]%32 == 0, 'Multiples of 32 required'
        boxed_image = letterbox_image(image, tuple(reversed(model_image_size)))
    else:
        new_image_size = (image_w - (image_w % 32), image_h - (image_h % 32))
        boxed_image = letterbox_image(image, new_image_size)
    image_data = np.array(boxed_image, dtype='float32')
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)
    return image_data

In [ ]:
def _get_feats(feats, anchors, num_classes, input_shape):
    num_anchors = len(anchors)
    anchors_tensor = np.reshape(np.array(anchors, dtype=np.float32), [1, 1, 1, num_anchors, 2])
    grid_size = np.shape(feats)[1:3]
    nu = num_classes + 5
    predictions = np.reshape(feats, [-1, grid_size[0], grid_size[1], num_anchors, nu])
    grid_y = np.tile(np.reshape(np.arange(grid_size[0]), [-1, 1, 1, 1]), [1, grid_size[1], 1, 1])
    grid_x = np.tile(np.reshape(np.arange(grid_size[1]), [1, -1, 1, 1]), [grid_size[0], 1, 1, 1])
    grid = np.concatenate([grid_x, grid_y], axis = -1)
    grid = np.array(grid, dtype=np.float32)

    box_xy = (1/(1+np.exp(-predictions[..., :2])) + grid) / np.array(grid_size[::-1], dtype=np.float32)
    box_wh = np.exp(predictions[..., 2:4]) * anchors_tensor / np.array(input_shape[::-1], dtype=np.float32)
    box_confidence = 1/(1+np.exp(-predictions[..., 4:5]))
    box_class_probs = 1/(1+np.exp(-predictions[..., 5:]))
    return box_xy, box_wh, box_confidence, box_class_probs


def correct_boxes(box_xy, box_wh, input_shape, image_shape):
    box_yx = box_xy[..., ::-1]
    box_hw = box_wh[..., ::-1]
    input_shape = np.array(input_shape, dtype = np.float32)
    image_shape = np.array(image_shape, dtype = np.float32)
    new_shape = np.around(image_shape * np.min(input_shape / image_shape))
    offset = (input_shape - new_shape) / 2. / input_shape
    scale = input_shape / new_shape
    box_yx = (box_yx - offset) * scale
    box_hw *= scale

    box_mins = box_yx - (box_hw / 2.)
    box_maxes = box_yx + (box_hw / 2.)
    boxes = np.concatenate([
        box_mins[..., 0:1],
        box_mins[..., 1:2],
        box_maxes[..., 0:1],
        box_maxes[..., 1:2]
    ], axis = -1)
    boxes *= np.concatenate([image_shape, image_shape], axis = -1)
    return boxes


def boxes_and_scores(feats, anchors, classes_num, input_shape, image_shape):
    box_xy, box_wh, box_confidence, box_class_probs = _get_feats(feats, anchors, classes_num, input_shape)
    boxes = correct_boxes(box_xy, box_wh, input_shape, image_shape)
    boxes = np.reshape(boxes, [-1, 4])
    box_scores = box_confidence * box_class_probs
    box_scores = np.reshape(box_scores, [-1, classes_num])
    return boxes, box_scores

In [ ]:
'''Draw detection frame'''
def draw_bbox(image, bboxes, classes):
    """
    bboxes: [x_min, y_min, x_max, y_max, probability, cls_id] format coordinates.
    """
    num_classes = len(classes)
    image_h, image_w, _ = image.shape
    hsv_tuples = [(1.0 * x / num_classes, 1., 1.) for x in range(num_classes)]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))

    random.seed(0)
    random.shuffle(colors)
    random.seed(None)

    for i, bbox in enumerate(bboxes):
        coor = np.array(bbox[:4], dtype=np.int32)
        fontScale = 0.5
        score = bbox[4]
        class_ind = int(bbox[5])
        bbox_color = colors[class_ind]
        bbox_thick = int(0.6 * (image_h + image_w) / 600)
        c1, c2 = (coor[0], coor[1]), (coor[2], coor[3])
        cv2.rectangle(image, c1, c2, bbox_color, bbox_thick)
    return image


def nms_boxes(boxes, scores):
    """Suppress non-maximal boxes.

    # Arguments
        boxes: ndarray, boxes of objects.
        scores: ndarray, scores of objects.

    # Returns
        keep: ndarray, index of effective boxes.
    """
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    areas = (x2-x1+1)*(y2-y1+1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)

        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w1 = np.maximum(0.0, xx2 - xx1 + 1)
        h1 = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w1 * h1

        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        inds = np.where(ovr <= 0.55)[0]  # threshold
        order = order[inds + 1]

    return keep

In [ ]:
def draw_boxes(image, boxes, scores, classes):
    _, ax = plt.subplots(1)
    ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    image_h, image_w, _ = image.shape

    for i, bbox in enumerate(boxes):
        [top, left, bottom, right] = bbox
        width, height = right - left, bottom - top
        center_x, center_y = left + width*0.5, top + height*0.5
        score, class_index = scores[i], classes[i]
        label = '{}: {:.4f}'.format(class_names[class_index], score)
        color = tuple([color/255 for color in colors[class_index]])
        ax.add_patch(Rectangle((left, top), width, height,
                               edgecolor=color, facecolor='none'))
        ax.annotate(label, (center_x, center_y), color=color, weight='bold',
                    fontsize=12, ha='center', va='center')
    return ax

In [ ]:
def evaluate(yolo_outputs, image_shape, class_names, anchors):
    score_thresh = 0.2
    anchor_mask = [[6, 7, 8], [3, 4, 5], [0, 1, 2]]
    boxes = []
    box_scores = []
    input_shape = np.shape(yolo_outputs[0])[1 : 3]
    input_shape = np.array(input_shape)*32

    for i in range(len(yolo_outputs)):
        _boxes, _box_scores = boxes_and_scores(
            yolo_outputs[i], anchors[anchor_mask[i]], len(class_names),
            input_shape, image_shape)
        boxes.append(_boxes)
        box_scores.append(_box_scores)
    boxes = np.concatenate(boxes, axis = 0)
    box_scores = np.concatenate(box_scores, axis = 0)

    mask = box_scores >= score_thresh
    boxes_ = []
    scores_ = []
    classes_ = []
    for c in range(len(class_names)):
        class_boxes_np = boxes[mask[:, c]]
        class_box_scores_np = box_scores[:, c]
        class_box_scores_np = class_box_scores_np[mask[:, c]]
        nms_index_np = nms_boxes(class_boxes_np, class_box_scores_np)
        class_boxes_np = class_boxes_np[nms_index_np]
        class_box_scores_np = class_box_scores_np[nms_index_np]
        classes_np = np.ones_like(class_box_scores_np, dtype = np.int32) * c
        boxes_.append(class_boxes_np)
        scores_.append(class_box_scores_np)
        classes_.append(classes_np)
    boxes_ = np.concatenate(boxes_, axis = 0)
    scores_ = np.concatenate(scores_, axis = 0)
    classes_ = np.concatenate(classes_, axis = 0)

    return boxes_, scores_, classes_

## Use Vart

In [ ]:
dpu = overlay.runner

In [ ]:
inputTensors = dpu.get_input_tensors()

In [ ]:
outputTensors = dpu.get_output_tensors()

In [ ]:
shapeIn = tuple(inputTensors[0].dims)

In [ ]:
shapeOut0 = (tuple(outputTensors[0].dims)) # (1, 13, 13, 75)
shapeOut1 = (tuple(outputTensors[1].dims)) # (1, 26, 26, 75)
shapeOut2 = (tuple(outputTensors[2].dims)) # (1, 52, 52, 75)

In [ ]:
outputSize0 = int(outputTensors[0].get_data_size() / shapeIn[0]) # 12675
outputSize1 = int(outputTensors[1].get_data_size() / shapeIn[0]) # 50700
outputSize2 = int(outputTensors[2].get_data_size() / shapeIn[0]) # 202800

In [ ]:
input_data = [np.empty(shapeIn, dtype=np.float32, order="C")]
output_data = [np.empty(shapeOut0, dtype=np.float32, order="C"),
               np.empty(shapeOut1, dtype=np.float32, order="C"),
               np.empty(shapeOut2, dtype=np.float32, order="C")]
image = input_data[0]

In [ ]:
def run2(display = False):
    orig_img_path = 'Data/test.png'

    # Read input image
    input_image = cv2.imread(orig_img_path)

    # Pre-processing
    image_size = input_image.shape[:2]
    image_data = np.array(pre_process(input_image, (416, 416)), dtype=np.float32)

    # Fetch data to DPU and trigger it
    image[0,...] = image_data.reshape(shapeIn[1:])
    job_id = dpu.execute_async(input_data, output_data)
    dpu.wait(job_id)

    # Retrieve output data
    conv_out0 = np.reshape(output_data[0], shapeOut0)
    conv_out1 = np.reshape(output_data[1], shapeOut1)
    conv_out2 = np.reshape(output_data[2], shapeOut2)
    yolo_outputs = [conv_out0, conv_out1, conv_out2]

    # Decode output from YOLOv3
    boxes, scores, classes = evaluate(yolo_outputs, image_size, class_names, anchors)

    if display:
        _ = draw_boxes(input_image, boxes, scores, classes)
    print("Number of detected objects: {}".format(len(boxes)))

    objectsDetected = []
    for i, bbox in enumerate(boxes):
        class_index = classes[i]
        objectsDetected.append(class_names[class_index])
    return(objectsDetected[0])

# Display to Adafruit IO through internet

In [ ]:
from pynq import GPIO
from time import sleep
from Adafruit_IO import MQTTClient
from Adafruit_IO import Client

ADAFRUIT_IO_USERNAME = ###
ADAFRUIT_IO_KEY = ###

In [ ]:
boxFeed1 = "stored-object"
boxFeed2 = "stored-object-2"
boxFeed3 = "stored-object-3"
retrieveFeed = "delete1"
scheduleFeed = "time"
newObjectFeed = "detect"
openBoxFeed = "open-box"
imageFeed = "stored-image"
getImageFeed = "get-image"

In [ ]:
def connected(client):
    print("Connected to Adafruit IO!")
    client.subscribe(boxFeed1)
    client.subscribe(boxFeed2)
    client.subscribe(boxFeed2)
    client.subscribe(retrieveFeed)
    client.subscribe(scheduleFeed)
    client.subscribe(newObjectFeed)
    client.subscribe(openBoxFeed)
    client.subscribe(imageFeed)
    client.subscribe(getImageFeed)

def message(client, feed_id, payload):
    print(f"Received: {feed_id} {payload}")

client = MQTTClient(ADAFRUIT_IO_USERNAME, ADAFRUIT_IO_KEY)
aio = Client(ADAFRUIT_IO_USERNAME, ADAFRUIT_IO_KEY)
client.on_connect = connected
client.on_message = message

# Open Camera for Video Streaming

In [ ]:
import cv2

camera = cv2.VideoCapture(0)

width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
camera.set(cv2.CAP_PROP_FRAME_WIDTH,width)
camera.set(cv2.CAP_PROP_FRAME_HEIGHT,height)

In [ ]:
import IPython

def imshow(img):
    returnValue, buffer = cv2.imencode('.jpg', img)
    IPython.display.display(IPython.display.Image(data=buffer.tobytes()))

# Main Loop

In [ ]:
# Declare Variables used in the main loop
from datetime import datetime
from PIL import Image as PIL_Image
from zoneinfo import ZoneInfo
import base64

boxObjects1, boxObjects2, boxObjects3 = [], [], []
scheduledTime = [] # in the form of [DD/MM, hh:mm, n, i]
i = 0
boxNumber = ""
objectIndex = ""
timeofDay = ""
Date = ""
newSchedule = ""
newRetrieval = ""
totalCount = 0

In [ ]:
def updateBox(N):
    output = ""
    if (N == 1):
        if (len (boxObjects1) == 0):
            aio.send_data(boxFeed1, "0")
        else:
            for i in (range(len(boxObjects1))):
                output += f"{i + 1}: {boxObjects1[i][0]}\n"
            aio.send_data(boxFeed1, output)
    elif (N == 2):
        if (len (boxObjects2) == 0):
            aio.send_data(boxFeed2, "0")
        else:
            for i in (range(len(boxObjects2))):
                output += f"{i + 1}: {boxObjects2[i][0]}\n"
            aio.send_data(boxFeed2, output)
    elif (N == 3):
        if (len (boxObjects3) == 0):
            aio.send_data(boxFeed3, "0")
        else:
            for i in (range(len(boxObjects3))):
                output += f"{i + 1}: {boxObjects3[i][0]}\n"
            aio.send_data(boxFeed3, output)


In [ ]:
def doRetrieve(boxNumber, objectIndex): # base index = 1, strings
    if (boxNumber == '1'):
        del(boxObjects1[int(objectIndex) - 1])
    if (boxNumber == '2'):
        del(boxObjects2[int(objectIndex) - 1])
    if (boxNumber == '3'):
        del(boxObjects3[int(objectIndex) - 1])
    updateBox(int(boxNumber))
    aio.send_data(openBoxFeed, int(boxNumber))

# Initialisation

In [ ]:
for i in range (1, 4):
    updateBox(i)

aio.send_data(retrieveFeed, "0")
aio.send_data(scheduleFeed, "0")
aio.send_data(newObjectFeed, "0")
aio.send_data(openBoxFeed, "0")
aio.send_data(imageFeed, "0")
aio.send_data(getImageFeed, "0")

In [ ]:
while True:
    i = int(aio.receive(newObjectFeed).value)
    if (i):
        num_frames = 40
        for _ in range(num_frames):
        # Capture frame-by-frame
            ret, frame_in = camera.read()
            if (not ret):
                # Release the Video Device if ret is false
                camera.release()
                # Message to be displayed after releasing the device
                print("Release camera resource")
                break
            imshow(frame_in)
            IPython.display.clear_output(wait=True)
        cv2.imwrite('Data/test.png', frame_in)
        result = run2(display=True)  # Using the photo for object detection

        img = cv2.imread("Data/test.png")
        resized_img = cv2.resize(img, (240, 180), interpolation=cv2.INTER_AREA) # Resize the taken photo to a smaller resolution for data transfer
        cv2.imwrite("Data/store.png", resized_img)
        with open("Data/store.png", "rb") as image_file:
            binary_data = image_file.read()
        encoded_bytes = base64.b64encode(binary_data)
        base64_string = encoded_bytes.decode("utf-8") # Encode the photo into Base64 for passing to the Adafruit IO

        boxObjects1.append([result, base64_string])  # Adding the object [object name, photo (base64)] to memory list
        updateBox(1)  # Updating the value shown on the Adafruit IO
        aio.send_data(openBoxFeed, 1)  # Tell the Arduino to open the box
        print(result)
        aio.send_data(newObjectFeed, 0)  # Resetting

    newSchedule = aio.receive(scheduleFeed).value #See what time the user want to open (in the format of "n i hh:mm DD/MM")
    if (newSchedule != "0"):
        # If there is a scheduled time
        boxNumber, objectIndex, timeofDay, Date = newSchedule.split()
        scheduledTime.append([Date, timeofDay, boxNumber, objectIndex])
        scheduledTime.sort()  # Make it so that it sees which object should be taken first
        if (boxNumber == '1'):  # Update the box object information shown, adding the scheduled time
            boxObjects1[int(objectIndex) - 1][0] = boxObjects1[int(objectIndex) - 1][0] + f" ({Date}, {timeofDay})"
        elif (boxNumber == '2'):
            boxObjects2[int(objectIndex) - 1][0] = boxObjects2[int(objectIndex) - 1][0] + f" ({Date}, {timeofDay})"
        elif (boxNumber == '3'):
            boxObjects3[int(objectIndex) - 1][0] = boxObjects3[int(objectIndex) - 1][0] + f" ({Date}, {timeofDay})"
        updateBox(int(boxNumber))
        aio.send_data(scheduleFeed, "0")
        print(scheduledTime)

    # if the time meets the schedule
    timeNow = datetime.now(ZoneInfo("Asia/Hong_Kong")).strftime("%d/%m %H:%M")
    while (len(scheduledTime) > 0 and timeNow >= (scheduledTime[0][0] + " " + scheduledTime[0][1])): # Checks if it has reached the time
        print(scheduledTime[0][0] + " " + scheduledTime[0][1])
        doRetrieve(scheduledTime[0][2], scheduledTime[0][3])
        del(scheduledTime[0])

    newRetrieval = aio.receive(retrieveFeed).value # format = "n i"
    if (newRetrieval != "0"):
        # If there is a retrieval of an object
        boxNumber, objectIndex = newRetrieval.split()
        doRetrieve(boxNumber, objectIndex)
        aio.send_data(retrieveFeed, "0")

    getImage = aio.receive(getImageFeed).value
    if (getImage != "0"):
        boxNumber, objectIndex = getImage.split()
        if (boxNumber == '1'):
            aio.send_data(imageFeed, boxObjects1[int(objectIndex) - 1][1]) # retrieve the base64 photo stored in the list
        if (boxNumber == '2'):
            aio.send_data(imageFeed, boxObjects2[int(objectIndex) - 1][1])
        if (boxNumber == '3'):
            aio.send_data(imageFeed, boxObjects3[int(objectIndex) - 1][1])
        aio.send_data(getImageFeed, "0")

In [ ]:
camera.release()

In [ ]:
del overlay
del dpu